In [6]:
pwd

'/home/wsuser/work'

In [7]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_a52c887f20bf41468f32fc39f1d88cd8 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='t6e9lqUJeBBJJVCNi_ZIIR7I6T56JYT2qsy6A4yuErJd',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_a52c887f20bf41468f32fc39f1d88cd8.get_object(Bucket='plantdiseaseidentification-donotdelete-pr-ls9zwh6qazl9iv', Key='Veg-dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [8]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [9]:
pwd

'/home/wsuser/work'

In [10]:
ls

fruit-dataset/  fruit.h5  Veg-dataset/


# Apply ImageDataGenerator functionality to Train and Test set

# Preprocessing

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1)

In [13]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Veg-dataset/train_set',target_size=(128,128),batch_size=16,class_mode='categorical')


Found 11386 images belonging to 9 classes.


In [14]:
x_test = test_datagen.flow_from_directory('/home/wsuser/work/Veg-dataset/test_set',target_size=(128,128),batch_size=16,class_mode='categorical')


Found 3416 images belonging to 9 classes.


# Import the models

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Convolution2D,MaxPool2D,Flatten

# Initializing the models

In [16]:
model=Sequential()

# Add CNN Layers¶

In [17]:
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

In [18]:
x_train.class_indices

{'Pepper,_bell___Bacterial_spot': 0,
 'Pepper,_bell___healthy': 1,
 'Potato___Early_blight': 2,
 'Potato___Late_blight': 3,
 'Potato___healthy': 4,
 'Tomato___Bacterial_spot': 5,
 'Tomato___Late_blight': 6,
 'Tomato___Leaf_Mold': 7,
 'Tomato___Septoria_leaf_spot': 8}

# Add Pooling layer

In [19]:
model.add(MaxPool2D(pool_size=(2,2)))

# Add Flatten layer

In [20]:
model.add(Flatten())

# Add Dense Layer

In [21]:
model.add(Dense(300, kernel_initializer='uniform',activation='relu'))
model.add(Dense(150, kernel_initializer='uniform',activation='relu'))
model.add(Dense(75, kernel_initializer='uniform',activation='relu'))

# Add Output Layer

In [22]:
model.add(Dense(9,activation='softmax', kernel_initializer='uniform'))

# Compile the model

In [23]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Fit the Model

In [24]:
model.fit_generator(x_train,steps_per_epoch=89,validation_data=x_test,validation_steps=27,epochs=10)

/tmp/wsuser/ipykernel_1299/753566444.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=89,validation_data=x_test,validation_steps=27,epochs=10)


Epoch 1/10
89/89 [==============================] - 48s 537ms/step - loss: 2.0189 - accuracy: 0.2163 - val_loss: 93.7945 - val_accuracy: 0.3171
Epoch 2/10
89/89 [==============================] - 47s 522ms/step - loss: 1.6335 - accuracy: 0.3843 - val_loss: 299.1470 - val_accuracy: 0.3704
Epoch 3/10
89/89 [==============================] - 48s 541ms/step - loss: 1.4086 - accuracy: 0.4761 - val_loss: 164.6916 - val_accuracy: 0.4884
Epoch 4/10
89/89 [==============================] - 48s 535ms/step - loss: 1.2665 - accuracy: 0.5400 - val_loss: 397.7639 - val_accuracy: 0.2870
Epoch 5/10
89/89 [==============================] - 48s 542ms/step - loss: 1.0927 - accuracy: 0.6044 - val_loss: 456.7229 - val_accuracy: 0.3866
Epoch 6/10
89/89 [==============================] - 47s 530ms/step - loss: 0.9926 - accuracy: 0.6432 - val_loss: 928.6741 - val_accuracy: 0.3380
Epoch 7/10
89/89 [==============================] - 48s 535ms/step - loss: 0.9224 - accuracy: 0.6657 - val_loss: 885.2817 - val_acc

# Save the Model

In [25]:
model.save("vegetable.h5")

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
 dense (Dense)               (None, 300)               38102700  
                                                                 
 dense_1 (Dense)             (None, 150)               45150     
                                                                 
 dense_2 (Dense)             (None, 75)                11325     
                                                        

# Test the Model

In [27]:
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np

In [28]:
model = load_model("vegetable.h5")

In [29]:
index=['Pepper,_bell___Bacterial_spot','Pepper,_bell___healthy','Potato___Early_blight','Potato___Late_blight','Potato___healthy','Tomato___Bacterial_spot','Tomato___Late_blight','Tomato___Leaf_Mold','Tomato___Septoria_leaf_spot']

# Test Pepper Bell Bacterial Spot Class Images

In [30]:
img = image.load_img('/home/wsuser/work/Veg-dataset/test_set/Pepper,_bell___Bacterial_spot/ad921dec-e88f-41d8-9455-0880c69063fc___NREC_B.Spot 9216.JPG',target_size=(128,128))

In [33]:
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred = np.argmax(model.predict(x),axis=1)
print('the given image belogs to=',index[pred[0]])

the given image belogs to= Pepper,_bell___healthy
